## Project: VGGNet on 2D systolic array and mapping on Cyclone IV GX


### 1. Train VGGNet with a layer of ic=16 and oc=8 and 4 bit quanization to achieve > 90% accuracy:

In [1]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

from models import *

global best_prec
use_gpu = torch.cuda.is_available()
print('=> Building model...')
    
    
batch_size = 128
model_name = "VGG16_quant_multi"
model = VGG16_quant()
# print(model)

normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    adjust_list = [150, 225]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1        

#model = nn.DataParallel(model).cuda()
#all_params = checkpoint['state_dict']
#model.load_state_dict(all_params, strict=False)
#criterion = nn.CrossEntropyLoss().cuda()
#validate(testloader, model, criterion)

=> Building model...
Files already downloaded and verified
Files already downloaded and verified


In [2]:
PATH = "result/VGG16_quant_multi/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 9038/10000 (90%)



### 2. Data Generation:

In [3]:
class SaveOutput:
    def __init__(self):
        self.outputs = []
    def __call__(self, module, module_in):
        self.outputs.append(module_in)
    def clear(self):
        self.outputs = []  
        
######### Save inputs from selected layer ##########
save_output = SaveOutput()
i = 0

for layer in model.modules():
    i = i+1
    if isinstance(layer, QuantConv2d):
        print(i,"-th layer prehooked")
        layer.register_forward_pre_hook(save_output)             
####################################################

dataiter = iter(testloader)
images, labels = next(dataiter)
images = images.to(device)
out = model(images)

3 -th layer prehooked
7 -th layer prehooked
12 -th layer prehooked
16 -th layer prehooked
21 -th layer prehooked
25 -th layer prehooked
28 -th layer prehooked
33 -th layer prehooked
37 -th layer prehooked
41 -th layer prehooked
46 -th layer prehooked
50 -th layer prehooked
54 -th layer prehooked


In [4]:
# The convolutional layer with a input channel number of 8 and a output channel number of 8 
# has a index of 17 in the model.features
weight_q = model.features[17].weight_q
w_alpha = model.features[17].weight_quant.wgt_alpha
w_bit = 4

weight_int = weight_q / (w_alpha / (2**(w_bit-1)-1))
print(weight_int)

tensor([[[[-3.0000,  6.0000,  7.0000],
          [ 1.0000,  0.0000,  5.0000],
          [ 2.0000,  1.0000,  6.0000]],

         [[-7.0000, -4.0000, -0.0000],
          [-7.0000, -7.0000, -3.0000],
          [ 3.0000, -1.0000, -4.0000]],

         [[-4.0000, -3.0000, -7.0000],
          [-7.0000, -5.0000, -7.0000],
          [-5.0000, -5.0000, -7.0000]],

         ...,

         [[-4.0000, -6.0000,  7.0000],
          [-1.0000, -7.0000, -5.0000],
          [-7.0000, -5.0000, -2.0000]],

         [[ 1.0000, -7.0000, -7.0000],
          [-2.0000, -7.0000, -7.0000],
          [-7.0000, -2.0000, -7.0000]],

         [[ 4.0000, -7.0000, -6.0000],
          [ 1.0000, -7.0000,  5.0000],
          [ 7.0000, -7.0000, -1.0000]]],


        [[[-7.0000, -7.0000, -7.0000],
          [-7.0000, -7.0000, -7.0000],
          [-7.0000, -7.0000, -7.0000]],

         [[-7.0000, -7.0000, -7.0000],
          [-7.0000, -7.0000, -7.0000],
          [-7.0000, -7.0000, -7.0000]],

         [[ 7.0000,  7.0000,  7

In [5]:
act = save_output.outputs[5][0]
act_alpha  = model.features[17].act_alpha
act_bit = 4
act_quant_fn = act_quantization(act_bit)

act_q = act_quant_fn(act, act_alpha)

act_int = act_q / (act_alpha / (2**act_bit-1))
print(act_int)

tensor([[[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  2.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  1.0000,  0.0000,  5.0000],
          ...,
          [ 0.0000,  6.0000,  2.0000,  ...,  0.0000,  2.0000,  0.0000],
          [ 0.0000,  5.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  2.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[ 1.0000,  0.0000,  2.0000,  ...,  5.0000,  5.0000,  5.0000],
          [ 1.0000,  0.0000,  2.0000,  ...,  0.0000,  1.0000,  5.0000],
          [ 3.0000,  0.0000,  1.0000,  ...,  1.0000,  4.0000,  9.0000],
          ...,
          [ 3.0000,  3.0000,  5.0000,  ...,  3.0000,  6.0000,  3.0000],
          [ 4.0000,  0.0000,  1.0000,  ...,  3.0000,  4.0000,  1.0000],
          [ 4.0000,  2.0000,  1.0000,  ...,  2.0000,  2.0000,  2.0000]],

         [[ 0.0000,  0.0000,  7.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  

In [6]:
conv_int = torch.nn.Conv2d(in_channels = 16, out_channels=8, kernel_size = 3, padding=1)
conv_int.weight = torch.nn.parameter.Parameter(weight_int)
conv_int.bias = model.features[17].bias
output_int = conv_int(act_int)
output_recovered = output_int * (act_alpha / (2**act_bit-1)) * (w_alpha / (2**(w_bit-1)-1))
print(output_recovered)

tensor([[[[ -1.7833,  -5.5624,  -5.3651,  ...,  -6.1467,  -4.9022,  -3.1037],
          [ -4.7732,  -8.3246, -10.3811,  ...,  -7.6492,  -7.1635,  -3.3769],
          [ -5.6610,  -8.1273,  -8.6054,  ...,  -5.0539,  -6.3060,  -4.5910],
          ...,
          [ -3.3465,  -4.4393,  -4.8415,  ...,  -6.0708,  -7.3532,  -5.5472],
          [ -4.0143,  -5.4561,  -6.3136,  ...,  -5.3423,  -5.8052,  -4.9098],
          [ -3.7108,  -5.8659,  -6.1467,  ...,  -3.1568,  -3.4072,  -3.3086]],

         [[ -2.8685,  -3.0278,  -4.0902,  ...,  -1.4342,  -1.8061,  -1.9123],
          [ -2.3373,  -3.0278,  -2.6029,  ...,   2.9747,   0.2656,  -0.4781],
          [ -2.3373,  -2.9747,  -0.8499,  ...,   3.3996,   0.3718,  -0.4250],
          ...,
          [ -3.7184,  -4.8870,  -4.3558,  ...,  -6.3212,  -4.7276,  -3.1340],
          [ -4.5152,  -5.5244,  -4.1964,  ...,  -2.8685,  -1.2749,  -1.2749],
          [ -2.9747,  -2.8153,  -0.9561,  ...,  -1.3280,   0.0000,   0.1062]],

         [[  3.1872,   4.0902,

In [7]:
output_ref = save_output.outputs[6][0]
diff = F.relu(output_recovered) - output_ref
print(diff.sum())

tensor(0.0004, device='cuda:0', grad_fn=<SumBackward0>)


In [8]:
# act_int.size = torch.Size([128, 16, 8, 8])  <- batch_size, input_ch, ni, nj
a_int = act_int[0,:,:,:]  # pick only one input out of batch
# a_int.size() = [16, 8, 8]

# conv_int.weight.size() = torch.Size([8, 16, 3, 3])  <- output_ch, input_ch, ki, kj
w_int = torch.reshape(weight_int, (weight_int.size(0), weight_int.size(1), -1))  # merge ki, kj index to kij
# w_int.weight.size() = torch.Size([8, 16, 9])

padding = 1
stride = 1
array_size = 8 # row and column number

nig = range(a_int.size(1))  ## ni group [0,1,...7]
njg = range(a_int.size(2))  ## nj group

 
icg = range(int(w_int.size(1)))  ## input channel [0,...15]
ocg = range(int(w_int.size(0)))  ## output channel


kijg = range(w_int.size(2)) # [0, .. 8]
ki_dim = int(math.sqrt(w_int.size(2)))  ## Kernel's 1 dim size


######## Padding before Convolution #######
a_pad = torch.zeros(len(icg), len(nig)+padding*2, len(njg)+padding*2).cuda()
# a_pad.size() = [16, 8+2pad, 8+2pad]
a_pad[ :, padding:padding+len(nig), padding:padding+len(njg)] = a_int.cuda()
a_pad = torch.reshape(a_pad, (a_pad.size(0), -1))  ## mergin ni and nj index into nij
# a_pad.size() = [16, (8+2pad)*(8+2pad)]

In [9]:
###########################################

p_nijg = range(a_pad.size(1)) ## paded activation's nij group [0, ...10*10-1]

psum = torch.zeros(array_size, len(p_nijg), len(kijg)).cuda() 

for kij in kijg:       
    for nij in p_nijg:     # time domain, sequentially given input
        m = nn.Linear(array_size*2, array_size, bias=False)
        m.weight = torch.nn.Parameter(w_int[:,:,kij])
        psum[:, nij, kij] = m(a_pad[:,nij]).cuda()

In [10]:
import math

a_pad_ni_dim = int(math.sqrt(a_pad.size(1))) # 8 + 2*pad = 10

o_ni_dim = int((a_pad_ni_dim - (ki_dim- 1) - 1)/stride + 1) #10 - 2 - 1 + 1 = 8
o_nijg = range(o_ni_dim**2) # [0, 8*8-1]    
    
out = torch.zeros(len(ocg), len(o_nijg)).cuda()
  
   
### SFP accumulation ###
for o_nij in o_nijg: 
    for kij in kijg:  #[0, ... 8]
        out[:,o_nij] = out[:,o_nij] + \
        psum[:, int(o_nij/o_ni_dim)*a_pad_ni_dim + o_nij%o_ni_dim + int(kij/ki_dim)*a_pad_ni_dim + kij%ki_dim, kij]
                ## 2nd index = (int(o_nij/32)*34 + o_nij%32) + (int(kij/3)*32 + kij%3)

In [14]:
out_2D = torch.reshape(out, (out.size(0), o_ni_dim, -1)) # nij -> ni & nj
difference = (out_2D - output_int[0,:,:,:])
print(difference.abs().sum())

tensor(0.0001, device='cuda:0', grad_fn=<SumBackward0>)


### Input Generation:

In [13]:
bit_precision = 4
file = open('activation_multi.txt', 'w') #write to file
file.write('#time0row7[msb-lsb],time0row6[msb-lst],....,time0row0[msb-lst]#\n')
file.write('#time1row7[msb-lsb],time1row6[msb-lst],....,time1row0[msb-lst]#\n')
file.write('#................#\n')

for i in range(a_pad.size(1)):  # time step
    for j in range(a_pad.size(0)): # row #
        if j < 8:
            X_bin = '{0:04b}'.format(int(a_pad[7-j,i].item()+0.001))
            for k in range(bit_precision):
                file.write(X_bin[k])
        elif j == 8:
            file.write('\n')
            X_bin = '{0:04b}'.format(int(a_pad[15,i].item()+0.001))
            for k in range(bit_precision):
                file.write(X_bin[k])
        else:
            X_bin = '{0:04b}'.format(int(a_pad[23-j,i].item()+0.001))
            for k in range(bit_precision):
                file.write(X_bin[k])
        #file.write(' ')  # for visibility with blank between words, you can use
    file.write('\n')
file.close() #close file    

### Weight Generation:

In [16]:
### Complete this cell ### 

for kij in range(len(kijg)):
    W = w_int[:,:,kij]  # w_int[array col num, array row num, kij]  || w_int[output channel, input channel, kij]


    bit_precision = 4
    file = open('weight_multi_kij'+str(kij)+'.txt', 'w') #write to file
    file.write('#col0row7[msb-lsb],col0row6[msb-lst],....,col0row0[msb-lst]#\n')
    file.write('#col1row7[msb-lsb],col1row6[msb-lst],....,col1row0[msb-lst]#\n')
    file.write('#................#\n')

    for i in range(W.size(0)):  # col #
        for j in range(W.size(1)): # row #
            if j < 8:
                if W[i,7-j] >= 0:
                    W_bin = '{0:04b}'.format(int(W[i,7-j].item()+0.001))
                else:
                    if np.abs(W[i,7-j].item()) < 1:
                        W_bin = '{0:016b}'.format(0)
                    else:
                        W_bin = '{0:04b}'.format(-int(W[i,7-j].item()-0.001))
                        inverted = ''.join('1' if bit == '0' else '0' for bit in W_bin)
                        W_bin = '{0:04b}'.format(int(inverted, 2) + 1)
            elif j == 8:
                file.write('\n')
                if W[i,15] >= 0:
                    W_bin = '{0:04b}'.format(int(W[i,15].item()+0.001))
                else:
                    if np.abs(W[i,15].item()) < 1:
                        W_bin = '{0:016b}'.format(0)
                    else:
                        W_bin = '{0:04b}'.format(-int(W[i,15].item()-0.001))
                        inverted = ''.join('1' if bit == '0' else '0' for bit in W_bin)
                        W_bin = '{0:04b}'.format(int(inverted, 2) + 1)
            else:
                if W[i,23-j] >= 0:
                    W_bin = '{0:04b}'.format(int(W[i,23-j].item()+0.001))
                else:
                    if np.abs(W[i,23-j].item()) < 1:
                        W_bin = '{0:016b}'.format(0)
                    else:
                        W_bin = '{0:04b}'.format(-int(W[i,23-j].item()-0.001))
                        inverted = ''.join('1' if bit == '0' else '0' for bit in W_bin)
                        W_bin = '{0:04b}'.format(int(inverted, 2) + 1)
            for k in range(bit_precision):
                file.write(W_bin[k])
                #file.write(' ')  # for visibility with blank between words, you can use
        file.write('\n')
        
    file.close() #close file  

### Partial Sum Generation:

In [16]:
### Complete this cell ###
ic_tile_id = 0 
oc_tile_id = 0 

for kij in range(len(kijg)):
    psum_tile = psum[:,:,kij]  
    # psum[array_size, len(p_nijg), len(kijg)]


    bit_precision = 16
    file = open('psum_multi_kij'+str(kij)+'.txt', 'w') #write to file
    file.write('#time0col7[msb-lsb],time0col6[msb-lst],....,time0col0[msb-lst]#\n')
    file.write('#time1col7[msb-lsb],time1col6[msb-lst],....,time1col0[msb-lst]#\n')
    file.write('#................#\n')

    for i in range(psum_tile.size(1)):  # col #
        for j in range(psum_tile.size(0)): # row #
            if psum_tile[7-j,i] >= 0:
                P_bin = '{0:016b}'.format(int(psum_tile[7-j,i].item()+0.001))
            else:
                if np.abs(psum_tile[7-j,i].item()) < 1:
                    P_bin = '{0:016b}'.format(0)
                else:
                    P_bin = '{0:016b}'.format(-int(psum_tile[7-j,i].item()-0.001))
                    inverted = ''.join('1' if bit == '0' else '0' for bit in P_bin)
                    P_bin = '{0:016b}'.format(int(inverted, 2) + 1)
            for k in range(bit_precision):
                file.write(P_bin[k])        
#             file.write(' ')  # for visibility with blank between words, you can use
        file.write('\n')
    file.close() #close file 

### Output Generation:

In [17]:
bit_precision = 16
file = open('out_multi.txt', 'w') #write to file


for i in range(out.size(1)):  # onij
    for j in range(out.size(0)): # output channel #
        if out[7-j,i] >= 0:
            O_bin = '{0:016b}'.format(int(out[7-j,i].item()+0.001))
        else:
            O_bin = '{0:016b}'.format(0) # If the output is negeative, we set it to zeros because of the relu
        for k in range(bit_precision):
            file.write(O_bin[k])        
        #file.write(' ')  # for visibility with blank between words, you can use
    file.write('\n')
file.close() #close file 

### Address Generation:

In [18]:
from collections import defaultdict
acc_addr = defaultdict(list)
out_seq = defaultdict(list)

In [19]:
for o_nij in o_nijg: 
    for kij in kijg:  #[0, ... 8]
        nij_idx = int(o_nij/o_ni_dim)*a_pad_ni_dim + o_nij%o_ni_dim + int(kij/ki_dim)*a_pad_ni_dim + kij%ki_dim
        out_seq[o_nij].append((nij_idx,kij))

In [20]:
for o_nij in out_seq:
    for pos in out_seq[o_nij]: # pos = (nij, kij)
        acc_addr[o_nij].append(pos[0] + pos[1] * (a_pad_ni_dim**2)) # nij + a_pad_dim (100) * kij

In [21]:
bit_precision = 11
file = open('acc_address_multi.txt', 'w') #write to file

for o_nij in acc_addr:
    for addr in acc_addr[o_nij]:
        A_bin = '{0:011b}'.format(int(addr+0.001))
        for k in range(bit_precision):
            file.write(A_bin[k])
        file.write('\n')
file.close() #close file 